## Korean text preprocessing

Korean Hate Speech Dataset (https://github.com/kocohub/korean-hate-speech)

9,381 human-labeled comments in total

7,896 training set, 471 validation set, and 974 test set

Each comment is annotated on two aspects, the existence of social bias and hate speech, given that hate speech is closely related to bias.

- hate speech classification: hate, offensive, none

In [1]:
# download the hate speech dataset
!git clone https://github.com/kocohub/korean-hate-speech.git

Cloning into 'korean-hate-speech'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 112 (delta 4), reused 0 (delta 0), pack-reused 103
Receiving objects: 100% (112/112), 93.18 MiB | 17.12 MiB/s, done.
Resolving deltas: 100% (48/48), done.
Checking out files: 100% (20/20), done.


In [2]:
# read text files
import pandas as pd
train_data = pd.read_csv('korean-hate-speech/labeled/train.tsv', sep='\t')
test_data = pd.read_csv('korean-hate-speech/labeled/dev.tsv', sep='\t')

In [3]:
train_data

,comments,contain_gender_bias,bias,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate
...,...,...,...,...
7891,힘내세요~ 응원합니다!!,False,none,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,False,none,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,False,none,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,False,none,none


In [4]:
test_data

,comments,contain_gender_bias,bias,hate
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,False,none,none
1,지현우 나쁜놈,False,none,offensive
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,False,none,hate
3,설마 ㅈ 현정 작가 아니지??,True,gender,hate
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,False,none,offensive
...,...,...,...,...
466,지현우 범죄 저지르지 않았나요?,False,none,offensive
467,여자인생 망칠 일 있나 ㅋㅋ,True,gender,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,False,others,offensive
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",True,gender,hate


- on-line, user-generated text data
- informal stylistic properties: 이모티콘, 비속어, 특수문자, 띄어쓰기/철자 오류, 일반적으로 사용되지 않는 단어 등

이러한 특성을 고려해, 텍스트를 torchtext/모델에 넣기 전 cleansing, tokenization 등을 적절히 수행한다.

### 1. 문장 내 한자, 특수기호 제거

hanja 라이브러리, 정규표현식 이용

In [6]:
import re
!pip install hanja
import hanja

     |████████████████████████████████| 122kB 3.8MB/s 
     |████████████████████████████████| 266kB 18.4MB/s 
     |████████████████████████████████| 245kB 32.1MB/s 
  Created wheel for hanja: filename=hanja-0.13.3-cp37-none-any.whl size=128426 sha256=cbcdbbb0d09f745cdb41ebc1ed65895f790ccdbd4c8dbf90eac338ae6b5a552f
  Stored in directory: /root/.cache/pip/wheels/45/fc/c9/b7e7cb5c86935a1a99e2ad07f763728f8f17560e7b815a4b27
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp37-cp37m-linux_x86_64.whl size=44117 sha256=5379d6dab8d175ffe614b7585e9d540b32604fd05747c9435adeb41d4d788792
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built hanja pyyaml
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,>=3

In [7]:
# 문장 내 한자 제거

sent = '이건진짜좋은映畫ㅋㅋ 라라랜드진짜재밌는영화!!!'

# 한자에 해당하는 유니코드 범위 검색
if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sent) is not None:
  # 한자를 한글로 변환
  sent_nohanja = hanja.translate(sent, 'substitution')

print(sent_nohanja)

이건진짜좋은영화ㅋㅋ 라라랜드진짜재밌는영화!!!


In [8]:
# 문장 내 특수기호 제거
    
sent = (r'\[[^\]]+\]|\([^\)]+\)|\<[^\>]+\>', '', str(sent_nohanja))
sent = re.sub("[.,!?||]", " ", str(sent))
sent = re.sub(r'[^가-힣|\s]', '', str(sent))
sent = re.sub(r'\s+', ' ', str(sent)) 

sent = sent.strip()

print(sent)

이건진짜좋은영화 라라랜드진짜재밌는영화


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Possible set union at position 5
  after removing the cwd from sys.path.


### 2. 띄어쓰기 오류 수정

자동화된 방식 시도

#### 2-1. soyspacing (https://github.com/lovit/soyspacing)

말뭉치를 가지고 직접 모델을 학습해야 함 (제공되는 모델 없음)

In [9]:
!pip install soyspacing
!git clone https://github.com/lovit/soyspacing.git

Cloning into 'soyspacing'...
remote: Enumerating objects: 270, done.
remote: Total 270 (delta 0), reused 0 (delta 0), pack-reused 270
Receiving objects: 100% (270/270), 2.13 MiB | 16.65 MiB/s, done.
Resolving deltas: 100% (130/130), done.


In [10]:
from soyspacing.countbase import CountSpace

# sample corpus for training: 15,602 normalized movie reviews
corpus_fname = 'soyspacing/demo_model/134963_norm.txt'
# 말뭉치는 더 많은, 더 다양한 문장들로 구성되어 있을수록 더 도움이 될 것임

model = CountSpace()
model.train(corpus_fname) # 말뭉치를 이용해서 모델 학습

all tags length = 694236 --> 57795, (num_doc = 15602)

In [11]:
# 학습한 모델 저장
model.save_model('sample_model', json_format=False)

In [12]:
# 학습한 모델 불러와서 사용
model = CountSpace()
model.load_model('sample_model', json_format=False)

In [13]:
# 띄어쓰기 교정을 위한 parameters
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

# 문장 띄어쓰기 교정 예시
# sent: "이건진짜좋은영화 라라랜드진짜좋은영화"

# with parameters
sent_corrected, tags = model.correct(
    doc=sent,
    verbose=verbose,
    force_abs_threshold=ft,
    nonspace_threshold=nt,
    space_threshold=st,
    min_count=mc)

print(sent_corrected)

이건 진짜 좋은 영화 라라랜드진짜 재밌는 영화


In [14]:
# 문장 띄어쓰기 교정 예시
# without parameters: default value를 이용
sent_corrected, tags = model.correct(sent)

print(sent_corrected)
print(tags)

이건 진짜 좋은 영화 라라랜드진짜 재밌는 영화
[0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, None, None, 1, 0, 0, 1, 0, 1]


특정 단어, 혹은 어절의 앞 뒤를 반드시 띄거나 붙여쓴다는 규칙이 있다면 이를 적용할 수 있음

어절과 어절 앞, 뒤에 대한 띄어쓰기 태그가 포함되어 있는 텍스트 파일을 준비 -> 'rule'로서 작용하도록


In [15]:
# rule example
# 진짜 101: 진짜 라는 단어의 앞, 뒤는 반드시 띄어쓰기를 하고, 진과 짜 사이에는 반드시 붙여쓰기를 한다는 의미

rule_f = open('rules.txt', 'w')
rule_f.write("가령	101" + "\n")
rule_f.write("진짜	101" + "\n")
rule_f.write("가게는	1001" + "\n")
rule_f.write("가게로	1001" + "\n")
rule_f.write("가게야	1001" + "\n")
rule_f.close()

print(open('rules.txt').readlines())

['가령\t101\n', '진짜\t101\n', '가게는\t1001\n', '가게로\t1001\n', '가게야\t1001\n']


In [16]:
from soyspacing.countbase import RuleDict

rule_dict = RuleDict('rules.txt')
sent_corrected, tags = model.correct(sent, rules=rule_dict)
print(sent_corrected)

이건 진짜 좋은 영화 라라랜드 진짜 재밌는 영화


#### 2-2. PyKoSpacing (https://github.com/haven-jeon/PyKoSpacing)

대용량 코퍼스를 학습하여 만들어진 띄어쓰기 딥 러닝 모델 제공

In [17]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-683s4rpx
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-683s4rpx
  Created wheel for pykospacing: filename=pykospacing-0.5-cp37-none-any.whl size=2255825 sha256=3e294921b5ed921df6fd9bdf36837660f2a1e77d919ce33948541fdbceedd341
  Stored in directory: /tmp/pip-ephem-wheel-cache-x9n6865o/wheels/4d/45/58/e26cb2b7f6a063d234158c6fd1e5700f6e15b99d67154340ba
Successfully built pykospacing


In [18]:
from pykospacing import Spacing
spacing = Spacing()
s = spacing('너무재밌었다그래서보는것을추천한다')
print(s)

너무 재밌었다 그래서 보는 것을 추천한다


#### 2-3. PyHanSpell (https://github.com/ssut/py-hanspell)

띄어쓰기 수정 + 맞춤법 수정 (네이버 한글 맞춤법 검사기 기반)

In [19]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-r942iqky
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-r942iqky
  Created wheel for py-hanspell: filename=py_hanspell-1.1-cp37-none-any.whl size=4871 sha256=71e04461e7c677930e84f0fc193b52dcf683e561dadb5c38304910660cb7616d
  Stored in directory: /tmp/pip-ephem-wheel-cache-r7lyxoau/wheels/0a/25/d1/e5e96476dbb1c318cc26c992dd493394fe42b0c204b3e65588
Successfully built py-hanspell


In [20]:
from hanspell import spell_checker
spelled_sent = spell_checker.check('외않됀데')
print(spelled_sent)

Checked(result=True, original='외않됀데', checked='왜 안 된대', errors=1, words=OrderedDict([('왜', 1), ('안', 1), ('된대', 1)]), time=0.7958540916442871)


### 3. Tokenization

In [21]:
!pip install konlpy
from konlpy.tag import Okt
okt = Okt()

     |████████████████████████████████| 19.4MB 244kB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 450kB 34.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


koNLPy Okt (Twitter, Open Korean Text) tokenizer

- 다른 형태소 분석기에 비해 단순한 품사 분류
- 비교적 가볍고 빠른 분석
- 사용자 사전 항목 추가 지원



In [22]:
print(okt.morphs('이건 진짜 재밌는 영화ㅋㅋㅋ 그래서 추천!!!'))

['이건', '진짜', '재밌는', '영화', 'ㅋㅋㅋ', '그래서', '추천', '!!!']


In [23]:
print(okt.nouns('이건 진짜 재밌는 영화ㅋㅋㅋ 그래서 추천!!!'))

['이건', '진짜', '영화', '추천']


In [24]:
# 특정 품사에 해당하는 토큰들만 선택하여 모델 인풋 텍스트를 구성하기도 함
# 예: 명사(Noun), 동사(Verb), 형용사(Adjective), 부사(Adverb) 등

adjectives = []

for token in okt.pos('이건 진짜 재밌는 영화ㅋㅋㅋ 그래서 추천!!!'):
  print(token)
  if token[1] == 'Adjective':
    adjectives.append(token[0])

print('adjectives:', adjectives)

('이건', 'Noun')
('진짜', 'Noun')
('재밌는', 'Adjective')
('영화', 'Noun')
('ㅋㅋㅋ', 'KoreanParticle')
('그래서', 'Adverb')
('추천', 'Noun')
('!!!', 'Punctuation')
adjectives: ['재밌는']


In [25]:
# 모델 인풋 텍스트에서 stopwords를 제거하기도 함
# stopwords: 유의미한 토큰만을 선별하기 위해 제거하는, 큰 의미가 없는 단어들

# stopword list example
# https://github.com/stopwords-iso/stopwords-ko
!git clone https://github.com/stopwords-iso/stopwords-ko.git

Cloning into 'stopwords-ko'...
remote: Enumerating objects: 29, done.
remote: Total 29 (delta 0), reused 0 (delta 0), pack-reused 29
Unpacking objects: 100% (29/29), done.


In [26]:
import json
with open('stopwords-ko/stopwords-ko.json') as json_file:
  stopwords_json = json.load(json_file)

In [27]:
print(stopwords_json)

['!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '...', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '<', '=', '>', '?', '@', '\\', '^', '_', '`', '|', '~', '·', '—', '——', '‘', '’', '“', '”', '…', '、', '。', '〈', '〉', '《', '》', '가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들', '게다가', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공동으로', '과', '과연', '관계가 있다', '관계없이', '관련이 있다', '관하여', '관한', '관해서는', '구', '구체적으로', '구토하다', '그', '그들', '그때', '그래', '그래도', '그래서', '그러나', '그러니', '그러니까', '그러면', '그러므로', '그러한즉', '그런 까닭에', '그런데', '그런즉', '그럼', '그럼에도 불구하고', '그렇게 함으로써', '그렇지', '그렇지 않다면', '그렇지 않으면', '그렇지만', '그렇지않으면', '그리고', '그리하여', '그만이다', '그에 따르는', '그위에', '그저', '그중에서', '그치지 않다', '근거로', '근거하여', '기대여', '기점으로', '기준으로', '기타', '까닭으로', '까악', '까지', '까지 미치다', '까지도', '꽈당', '끙끙', '끼익', '나', '나머지는', '남들', '남짓', '너', '너희', '너희들', '네', '넷', '년', '논하지 않다', '놀라다', '누가 알겠는가', '누구', '다른', '다른 방면으로', '

In [28]:
sent_filtered = []

for token in okt.morphs('이건 진짜 재밌는 영화ㅋㅋㅋ 그래서 추천!!!'):
  if token in stopwords_json:
    pass
  else:
    sent_filtered.append(token)

print(' '.join(sent_filtered))


이건 진짜 재밌는 영화 ㅋㅋㅋ 추천 !!!


### 위의 전처리 과정들을 함수로 정리

In [29]:
def cleanse_sentence(sent): 
    
    # 문장 내 한자 제거
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sent) is not None:
        sent = hanja.translate(sent, 'substitution')
        
    # 문장 내 특수기호 제거
    sent = (r'\[[^\]]+\]|\([^\)]+\)|\<[^\>]+\>', '', str(sent))
    sent = re.sub("[.,!?||]", " ", str(sent))
    sent = re.sub(r'[^가-힣|\s]', '', str(sent))
    sent = re.sub(r'\s+', ' ', str(sent)) 
    
    return sent.strip()


# 문장 토큰화 함수
def tokenizer(string):
    okt = Okt()

    # 위 함수를 적용하여 불필요한 한자, 특수기호 등 제거
    string = cleanse_sentence(string)
    # 미처 제거되지 못한 특수기호 마저 제거
    string.replace("…","")
    string.replace("ㆍ","")

    # 띄어쓰기 오류 수정
    # from pykospacing import Spacing
    # spacing = Spacing()
    string_corrected = spacing(string)

    # 형태소 분석기를 적용하여 문장 토큰화
    morphs = okt.morphs(string_corrected)

    # 분석된 토큰들 중 stopwords에 포함되는 것을 제거
    # stopword list: https://github.com/stopwords-iso/stopwords-ko/blob/master/stopwords-ko.json
    morphs_filtered = []
    for morph in morphs:
      if morph not in stopwords_json:
        morphs_filtered.append(morph)
    
    return morphs_filtered
        

In [37]:
# raw text example
for text in train_data['comments'][500:515]:
  print(text)

결국 했지만 몰랐다 인가요?!
결국엔 양세종이나 장혁...둘중에 하난 죽거나 아님 둘다 살거나...뭐 이성계쪽은 다 죽겠지...그래도 역사는 못바꾸겠지...
결국은 주말시간대에 20프로 실패했구나 역시 스브스는 평일시간대에 21프로 돌파한 김사부2가 최고네 김사부2는 초중반에 21프로 돌파중인데ㄷㄷㄷ 김사부2 한석규 대상 확정 ㄷㄷㄷ
결론은 많이 먹어서..예요
결론은 빠순이들이 문제라는거 아냐
결백은 당신에게 안어울림 그냥 자백하고 반성하는 척이라도 해라
결별관심없고, 그냥 한혜진 왜나오는지 모르겠음 화사가 차라리 백번 착해보이고 궁금함
결헌식 꼭 하고 축하 받으세요
결혼 가즈아~~~♡♡♡겉껍데기(외모+돈)을 좋아하는 사람들이 많은 이 세상에 마음이나 성격을 보고 사람을 선택할 줄 아는 눈을 가진 세아씨가 멋지고 부럽네요.
결혼 많이해서 좋으시겟다
결혼 못하는놈들 댓글보소 ㅋㅋㅋ
결혼 잘 해서 행복한듯 보기가 좋내요.
결혼 축하합니다~ 행복하세요
결혼까지 생각했어 같은집 같은방에서 슬퍼도 bye bye bye~~~~#
결혼까지는 아닌듯연애하다말듯여자가 너무아까운 나이자나


In [38]:
# 전처리 함수 적용 예시
for text in train_data['comments'][500:515]:
  print(tokenizer(text))

['했지만', '몰랐다', '인가요']
['엔', '양세종', '이나', '장혁', '중', '난', '죽거나', '아님', '다', '살거나', '뭐', '성계', '쪽', '은', '다', '죽겠지', '역사', '는', '못', '바꾸겠지']
['은', '주말', '시간대', '프로', '실패했구나', '역시', '스브스', '는', '평일', '시간대', '프로', '돌파', '한', '김사', '부가', '최고', '김', '사부', '는', '초', '중반', '프로', '돌파', '중', '인데', '김', '사부', '한석규', '대상', '확정']
['결론', '은', '많이', '먹어서', '예요']
['결론', '은', '빠순이', '문제', '라는', '거', '아냐']
['결백', '은', '안', '어울림', '그냥', '자백', '하고', '반성', '하는', '척', '이라도', '해', '라']
['결별', '관심', '없고', '그냥', '한', '혜진', '나오는지', '모르겠음', '화사', '백', '번', '착해', '보이', '고', '궁금함']
['결', '헌', '식', '꼭', '하고', '축하', '받으세요']
['결혼', '즈', '겉', '껍데기', '외모', '돈', '좋아하는', '사람', '많은', '세상', '마음', '이나', '성격', '보고', '사람', '선택', '할', '줄', '아는', '눈', '가진', '세', '아씨', '멋지고', '부럽네요']
['결혼', '많이', '해서', '좋으시겟다']
['결혼', '못', '하는', '놈', '댓글', '보소']
['결혼', '잘', '해서', '행복한', '듯', '보기', '좋내요']
['결혼', '축하', '합니다', '행복하세요']
['결혼', '생각', '했어', '같은', '집', '같은', '방', '슬퍼도']
['결혼', '까지는', '아닌', '듯', '연애', '하다', '말듯', '여자', '너무', '아까운', '이자나

## torchtext를 이용해 데이터 세팅하기

### torchtext: PyTorch가 제공하는 텍스트 관련 기능들을 모은 라이브러리
* from torchtext.legacy import data, datasets
* from torchtext.vocab import Vectors 등
* 파일 로드하기, 토큰화(tokenization), 사전(vocab) 구성, 단어->정수(integer) 인코딩, 단어 벡터 구성, 배치화(batching) 등
* torchtext 내부 'data'에서 하위 메서드를 불러와서 사용할 것임 (TabularDataset, BucketIterator 등)

In [39]:
import torch, torchtext
# from torchtext import data, datasets
from torchtext.legacy import data, datasets

In [40]:
train_data

,comments,contain_gender_bias,bias,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,False,others,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,False,none,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",False,none,hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",False,none,none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,True,gender,hate
...,...,...,...,...
7891,힘내세요~ 응원합니다!!,False,none,none
7892,힘내세요~~삼가 고인의 명복을 빕니다..,False,none,none
7893,힘내세용 ^^ 항상 응원합니닷 ^^ !,False,none,none
7894,힘내소...연기로 답해요.나도 53살 인데 이런일 저런일 다 있더라구요.인격을 믿습...,False,none,none


In [56]:
# torchtext 필드 선언
# 필드: 데이터에 대해 앞으로 어떤 전처리를 할 것인지 정의하기 위한 도구
TEXT = data.Field(preprocessing=tokenizer, include_lengths=True, batch_first=True, sequential=False)
LABEL = data.LabelField(dtype=torch.long)

data_fields = [("comments", TEXT), ("contain_gender_bias", None), ("bias", None), ("hate", LABEL)]

# 데이터 불러오기
train_set, test_set = data.TabularDataset.splits(path ='korean-hate-speech/labeled', 
                                                   train='train.tsv',
                                                   test='dev.tsv',
                                                   format='tsv',
                                                   fields=data_fields,
                                                   skip_header=True
                                                  )

In [57]:
print('NUMBER of TRAIN data:', len(train_set))
print('NUMBER of TEST data:', len(test_set))

NUMBER of TRAIN data: 7896
NUMBER of TEST data: 471


In [61]:
print(vars(train_set.examples[1]))

{'comments': ['한국', '적', '인', '미인', '대표', '적', '인', '분', '너무나', '곱', '고', '아름다운', '모습', '모습', '뒤', '슬픔', '미', '처', '알', '지', '못', '했네요'], 'hate': 'none'}


In [62]:
# 분류 모델에서 사용할 수 있는 형태로 변환
BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits((train_set, test_set), 
                                                           sort_key=lambda x: len(x.comments),
                                                          sort_within_batch=False,
                                                           repeat=False,
                                                          batch_size=BATCH_SIZE, 
                                                           device=device)

iterator 내에서 batch를 이용해 model prediction을 얻고 model training, evaluation 등을 수행
- 실제 모델에 적용하는 것은 한국어 뉴스기사 분류 실습에서 다룰 예정